In [ ]:
__nbid__ = '0040'
__author__ = 'Brian Merino <brian.merino@noirlab.edu>, Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20250912' # yyyymmdd; version datestamp of this notebook
__keywords__ = ['gmos','gemini','stars','dragons']

# Gemini GMOS starry field photometry reduction using DRAGONS Python API
#### adapted from https://dragons.readthedocs.io/projects/gmosimg-drtutorial/en/stable/ex1_gmosim_starfield_api.html

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [Prepare the working directory](#Prepare)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Bad Pixel Mask](#BPM)
* [Create Master Bias](#Master_Bias)
* [Create Master Flat Field](#Master_Flat)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to reduce GMOS imaging data using the Gemini DRAGONS package on the Data Lab science platform using a custom DRAGONS kernel `"DRAGONS-4.0.0 (DL,Py3.12)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing bias, flats, fringe, and science frames, and creating a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It can currently be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, as well as spectroscopic data taken with GNIRS, GHOST, and GMOS in longslit mode. Linked <a href="https://dragons.readthedocs.io/en/stable/">here</a> is a general list of guides, manuals, and tutorials about the use of DRAGONS.

The DRAGONS kernel has been made available in the Data Lab environment, allowing users to access the routines without being dependent on installing the software on their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to reduce example Gemini North GMOS I-band imaging data fully. This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS GMOS data reduction process; rather, it will just show one example of a standard reduction of a GMOS imaging dataset. 

The data used in this notebook example is GMOS I band imaging from the Gemini archive of a starry field from the Gemini North Hamamatsu CCD commissioning (Program: GN-2017A-SV-151).

<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

For this notebook specifically, please acknowledge:
* DRAGONS publication: Labrie et al., <a href="https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract">"DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South"</a>, ASPC, 523, 321L 

* <a href="https://zenodo.org/record/7776065#.ZDg5qOzMLUI">DRAGONS open source software publication</a>

<a class="anchor" id="imports"></a>
# Importing Python libraries

In [ ]:
import warnings
import glob
import os
import shutil

from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce

from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.exceptions import AstropyWarning

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

warnings.simplefilter('ignore', category=AstropyWarning)

<a class="anchor" id="Prepare"></a>
# Prepare the working directory

If you have any intermediate files that were created from running this code in the past, you will need to remove them from your working directory. The cell below defines a clean-up function that will remove all the fits files from your working directory. This function will be called again at the end of the tutorial, leaving you with only the final product. By default, this function will delete all files in the working directory. If there are files that have been previously reduced that you would like to keep, set `save_reduced=1` when calling the function.

In [ ]:
def clean_up(save_reduced=0):
    #Does the calibrations directory already exist?
    caldb_Exist = os.path.exists('./calibrations') 
    
    if caldb_Exist:
        shutil.rmtree('./calibrations', ignore_errors=True)

    #Remove existing log and list files.
    work_dir_path = os.getcwd()
    work_dir = os.listdir(work_dir_path)

    for item in work_dir:
        if item.endswith(".log") or item.endswith(".list"):
            os.remove(os.path.join(work_dir_path, item))
    
    #Next, we will remove all the existing fits files, except for the previously reduced files, depending on what you set save_reduced to.
    if save_reduced:
        all_files_0 = glob.glob('*.fits')
        save = dataselect.select_data(all_files_0, [], ['PROCESSED'])
        
        for s in save:
            os.remove(os.path.join(work_dir_path,s))

    else:
        all_files_0 = glob.glob('*.fits')
        for a in all_files_0:
            os.remove(os.path.join(work_dir_path,a))

In [ ]:
clean_up(save_reduced=0)

<a class="anchor" id="About"></a>
# About the dataset

The data used for this tutorial is a dithered sequence on a starry field.

The table below contains a summary of the dataset:


| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | N20170614S0201-205 | 10 s, i-band |
| Bias | N20170613S0180-184 |  |
| Bias | N20170615S0534-538 |  |
| Twilight Flats | N20170702S0178-182 | 40 to 16 s, i-band |
| BPM | bpm_20170306_gmos-n_Ham_22_full_12amp.fits |  |

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data

Downloading I-band images from the Gemini archive to the current working directory. This step only needs to be executed once.

If you run this notebook for the first time and need to download the dataset, set the variable "download=True". The notebook will not redownload the dataset if it is set to False. This will become particularly useful if you run the notebooks more than once. 

In [ ]:
%%bash 

# create file that lists FITS files to be downloaded
echo "\
http://archive.gemini.edu/file/N20170613S0180.fits
http://archive.gemini.edu/file/N20170613S0181.fits
http://archive.gemini.edu/file/N20170613S0182.fits
http://archive.gemini.edu/file/N20170613S0183.fits
http://archive.gemini.edu/file/N20170613S0184.fits
http://archive.gemini.edu/file/N20170614S0201.fits
http://archive.gemini.edu/file/N20170614S0202.fits
http://archive.gemini.edu/file/N20170614S0203.fits
http://archive.gemini.edu/file/N20170614S0204.fits
http://archive.gemini.edu/file/N20170614S0205.fits
http://archive.gemini.edu/file/N20170615S0534.fits
http://archive.gemini.edu/file/N20170615S0535.fits
http://archive.gemini.edu/file/N20170615S0536.fits
http://archive.gemini.edu/file/N20170615S0537.fits
http://archive.gemini.edu/file/N20170615S0538.fits
http://archive.gemini.edu/file/N20170702S0178.fits
http://archive.gemini.edu/file/N20170702S0179.fits
http://archive.gemini.edu/file/N20170702S0180.fits
http://archive.gemini.edu/file/N20170702S0181.fits
http://archive.gemini.edu/file/N20170702S0182.fits
http://archive.gemini.edu/file/bpm_20170306_gmos-n_Ham_22_full_12amp.fits\
" > gmos_im_star.list

In [ ]:
%%bash

download="True"

if [ $download == "True" ]; then
    wget --no-check-certificate -N -q -i gmos_im_star.list

else
    echo "Skipping download. To download the data set used in this notebook, set download=True."
fi

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This tells the system where to put the calibration database. This database will keep track of the processed calibrations we are going to send to it.

In [ ]:
logutils.config(file_name='gmos_data_reduction.log')
caldb = cal_service.set_local_database()
caldb.init("w")

In [ ]:
all_files = glob.glob('N2017*[0-9].fits')
all_files.sort()

<a class="anchor" id="File_Lists"></a>
# Create file lists

This data set contains science and calibration frames. For some programs, it could have different observed targets and exposure times depending on how you organize your raw data.

The DRAGONS data reduction pipeline does not organize the data for you. You have to do it. DRAGONS provides tools to help you with that.

The first step is to create lists that will be used in the data reduction process. For that, we use dataselect. Please refer to the [dataselect](https://dragons.readthedocs.io/projects/recipe-system-users-manual/en/stable/supptools/dataselect.html?highlight=dataselect) documentation for details regarding its usage.

**List of biases**

In [ ]:
list_of_biases = dataselect.select_data(
    all_files,
    ['BIAS'],
    []
)

**List of flats**

If your dataset has flats obtained with more than one filter, you can add the --expr 'filter_name=="i"' expression to get only the flats obtained within the i-band. For example:

In [ ]:
list_of_flats = dataselect.select_data(
     all_files,
     ['FLAT'],
     [],
     dataselect.expr_parser('filter_name=="i"')
)

**List of science data**

In [ ]:
list_of_science = dataselect.select_data(
    all_files,
    [],
    ['CAL'],
    dataselect.expr_parser('(observation_class=="science" and filter_name=="i")')
)

<a class="anchor" id="BPM"></a>
## **Bad pixel mask**

Starting with DRAGONS v3.1, the static bad pixel masks (BPMs) are now handled as calibrations. They are downloadable from the archive instead of being packaged with the software. They are automatically associated like any other calibrations. This means that the user now must download the BPMs along with the other calibrations and add the BPMs to the local calibration manager.

In [ ]:
for bpm in dataselect.select_data(all_files, ['BPM']):
    caldb.add_cal(bpm)

<a class="anchor" id="Master_Bias"></a>
# Create a master bias
We start the data reduction by creating a master bias for the science data. It can be created and added to the calibration database using the commands below. The master bias will have the name of the first bias with the suffix _bias.fits

In [ ]:
reduce_bias = Reduce()
reduce_bias.files.extend(list_of_biases)
reduce_bias.runr()

<a class="anchor" id="Master_Flat"></a>
# Create a master flat field

Twilight flat images are used to produce an imaging master flat and the result is added to the calibration database.

The master flat will have the name of the first twilight flat file with the suffix _flat.fits.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_of_flats)
reduce_flats.runr()

<a class="anchor" id="Reduce_Science"></a>
# Reduce science images

Once our calibration files are processed and added to the database, we can run reduce on our science data.

This command will generate bias and flat corrected files and will stack them. If a fringe frame is needed, this command will apply the correction. The stacked image will have the _stack suffix.

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the "SCI" extension, the variance is in the "VAR" extension, and the data quality plane (mask) is in the "DQ" extension.

Each reduced science image will have the original name with the suffix _image.fits.

In [ ]:
reduce_science = Reduce()
reduce_science.files.extend(list_of_science)
reduce_science.runr()

<a class="anchor" id="Display_Image"></a>
# Display the stacked image

In [ ]:
image_file = "N20170614S0201_image.fits"
hdu_list = fits.open(image_file)
wcs = WCS(hdu_list[1].header)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(projection=wcs)
plt.imshow(image_data,cmap='gray',norm=LogNorm(vmin=0.01, vmax=1000000),origin='lower')
plt.xlabel('Right Ascension [hh:mm:ss]',fontsize=14,fontweight='bold')
plt.ylabel('Declination [degree]',fontsize=14,fontweight='bold')
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove duplicate calibrations and remove raw data (uncomment lines before running)

In [ ]:
#clean_up(save_reduced=1)